In [2]:
""" HW3 - Exercise 1.2 """

import paho.mqtt.client as mqtt
import redis
import json

# Create a new MQTT client
client = mqtt.Client()

#Connection to Redis
REDIS_HOST = 'redis-10596.c135.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 10596
REDIS_USERNAME = 'default'
REDIS_PASSWORD = 'iW4IWt7Jp1V6tdd6ZggZzwKRQDEp1nlE'

redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print('Redis Connected:', is_connected)

Redis Connected: True


In [3]:
# Define the callbacks for when the client receives a response from the server
# and when a message is published on a subscribed topic
def on_connect(client, userdata, flags, rc):
    print(f'Connected with result code {str(rc)}')
    # Subscribe to a topic when the client connects
    client.subscribe('s316001')

def on_message(client, userdata, msg):
    message = msg.payload.decode()    #it is converted to a string
    message_dict = json.loads(message)   #it is converted to a dictionary
    mac_address = message_dict['mac_address']
    events_list = message_dict['events']

    #create the two timeseries for a given pc have not been created yet
    if not redis_client.exists(f'{mac_address}:battery'): 
        try:
            redis_client.ts().create(f'{mac_address}:battery')
        except redis.ResponseError:
            pass
        try:
            redis_client.ts().create(f'{mac_address}:power')
        except redis.ResponseError:
            pass 
    
    for event in events_list:
        timestamp_ms = event['timestamp']
        battery_level = event['battery_level']
        power_plugged = event['power_plugged']
        redis_client.ts().add(f'{mac_address}:battery', timestamp_ms, battery_level)
        redis_client.ts().add(f'{mac_address}:power', timestamp_ms, power_plugged)
    
# Set the callbacks
client.on_connect = on_connect
client.on_message = on_message

client.connect('mqtt.eclipseprojects.io', 1883)

0

In [4]:
# Start the client loop to process network events and call the callbacks
client.loop_forever()

Connected with result code 0


KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0ec8e575-19c8-4894-9160-01c0b36ec399' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>